In [513]:
import pandas as pd
import random
from keytotext import pipeline
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to C:\Users\M.J.
[nltk_data]     Rock\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [514]:
# Just uploading and merging 4 datasets from keggle:
# https://www.kaggle.com/datasets/shivamb/netflix-shows
# These are just listings of movies and tv shows from 4 different platforms

netflix = pd.read_csv("netflix_titles.csv")
hulu = pd.read_csv("hulu_titles.csv")
disney = pd.read_csv("disney_plus_titles.csv")
amazon = pd.read_csv("amazon_prime_titles.csv")

netflix['platform'] = 'netflix'
hulu['platform'] = 'hulu'
disney['platform'] = 'disney'
amazon['platform'] = 'amazon'

frames = [netflix, hulu, disney, amazon]
streaming = pd.concat(frames)
streaming.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,platform
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",netflix
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",netflix
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,netflix


In [515]:
# I decided to play around with description. It was broken down by words, punctionation andf irrelevant words was excluded, 
# everything was lowercased, and then put in a separate column

sw = nltk.corpus.stopwords.words('english') + ["a", "the"]

tokens = []

for index, row in streaming.iterrows():
    text = str(row[11])
    token = re.findall('\w+', text)
    token = [x.lower() for x in token]
    token = [x if x not in sw else None for x in token]
    while None in token:
        token.remove(None)
    tokens.append(token)

streaming['disc_words'] = tokens

streaming.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,platform,disc_words
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",netflix,"[father, nears, end, life, filmmaker, kirsten,..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",netflix,"[crossing, paths, party, cape, town, teen, set..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,netflix,"[protect, family, powerful, drug, lord, skille..."


In [516]:
# Then I grouped everything by genre and esssentially merged all keywords within a genre

streaming_sum = streaming.groupby('listed_in').agg({'disc_words': ['sum']})
streaming_sum

,disc_words
,sum
listed_in,
Action,"[cia, operative, exacts, revenge, mob, boss, k..."
Action & Adventure,"[young, cia, operative, matt, weston, must, ge..."
"Action & Adventure, Anime Features","[escaping, poverty, become, witcher, vesemir, ..."
"Action & Adventure, Anime Features, Children & Family Movies","[world, humans, fearsome, monsters, live, unea..."
"Action & Adventure, Anime Features, Classic Movies","[earth, federation, last, hope, fight, princip..."
...,...
Thrillers,"[deadly, home, invasion, couple, new, dream, h..."
Unscripted,"[66, teams, descend, upon, fiji, compete, epic..."


In [522]:
# Then I tried "keytotext", it's a library for generating sentences from keywords, it's not super impressive but I just wanted 
# to try NLP. We may consider digging deeper into that field. 

# This loop essentially chooses a random genre then picks a few keywords belonging to it and tries to generate a sentence out of 
# them. Again, not super impressive at the current state, but we might consider doing a deep dive into NLPs for our project.

nlp = pipeline("k2t-base")
config = {"do_sample": True, "num_beams": 4, "no_repeat_ngram_size": 3, "early_stopping": True}
    
for i in range(3): 
    ran_genre = random.choice(streaming_sum.index)
    ran_words = []
    
    for i in range(12):
        ran_words.append(random.choice(streaming_sum['disc_words']['sum'][ran_genre]))
        ran_genre, ran_words
    print("Genres: ", ran_genre, end = "\n\n")
    print("Keywords: ", ran_words, end = "\n\n")
    print("Attempt: ", nlp(ran_words, **config), end = "\n\n\n")

Genres:  Animation, Buddy, Comedy

Keywords:  ['huge', 'huge', 'two', 'timon', 'chipmunks', 'connection', 'named', 'two', 'trying', 'good', 'huge', 'little']

Attempt:  Timon Chipmunks played a little song in the episode titled "The Two


Genres:  Action, Animation, Drama

Keywords:  ['film', 'destroy', 'funeral', 'want', 'grandmother', 'new', 'award', 'bombarded', 'businessmen', 'city', 'greedy', 'york']

Attempt:  The film "Dombed" is from the city of Grety, New York


Genres:  Action-Adventure, Animation, Musical

Keywords:  ['cat', 'meet', 'young', 'meet', 'animated', 'oliver', 'star', 'york', 'explores', 'young', 'ferb', 'gang']

Attempt:  The young|ferb group has an average, animated, and oliver (star).


